In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
# from base import *
from joblib import Parallel, delayed
import random
import time

## TEST NB

In [3]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
import macposts
import MNM_mcnb
from mcDODE import *

In [5]:
origin_spc_folder = os.path.join(r'D:\documents\GitHub\MAC-POSTS\data',
                                 r'input_files_MckeesRocks_SPC_new')
cache_folder = os.path.join('cahce')

In [6]:
nb = MNM_mcnb.MNM_network_builder()

In [7]:
print(origin_spc_folder)
print(type(origin_spc_folder))

D:\documents\GitHub\MAC-POSTS\data\input_files_MckeesRocks_SPC_new
<class 'str'>


In [8]:
nb.load_from_folder(origin_spc_folder)

In [9]:
nb.config

[DTA]
network_name = Snap_graph
unit_time = 5
total_interval = 5000
assign_frq = 180
start_assign_interval = 0
max_interval = 24
flow_scalar = 2
num_of_link = 16110
num_of_node = 6297
num_of_O = 283
num_of_D = 283
OD_pair = 80089
adaptive_ratio_car = 0.2
adaptive_ratio_truck = 0.0
routing_type = Biclass_Hybrid

[STAT]
rec_mode = LRn
rec_mode_para = 2
rec_folder = record
rec_volume = 0
volume_load_automatic_rec = 0
volume_record_automatic_rec = 0
rec_tt = 1
tt_load_automatic_rec = 0
tt_record_automatic_rec = 0

[FIXED]
path_file_name = path_table
num_path = 216720
choice_portion = Buffer
buffer_length = 24
route_frq = 180

[ADAPTIVE]
route_frq = 180

In [10]:
(f_car, f_truck, loss) = pickle.load(open('28iteration.pickle', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: '28iteration.pickle'

In [11]:
path_len = len(nb.path_table.ID2path) * nb.config.config_dict['DTA']['max_interval']
# car_flow = np.random.rand(path_len) / 100
# truck_flow = np.zeros(path_len)

In [13]:
num_interval = nb.config.config_dict['DTA']['max_interval']
f_car = np.random.rand(num_interval * nb.config.config_dict['FIXED']['num_path']) * 300
f_truck = np.random.rand(num_interval * nb.config.config_dict['FIXED']['num_path']) * 30

In [14]:
nb.update_demand_path2(f_car, f_truck)

In [ ]:
pickle.dump((f_car * 2, f_truck * 2, loss), open('2_28iteration_double.pickle', 'wb'))

In [15]:
f_car

array([168.96689113,  50.52978051, 239.4669118 , ..., 255.17376694,
       270.65032149, 217.8284554 ])

In [16]:
(f_car * 2).sum()

np.float64(1560046511.1890197)

In [17]:
nb.dump_to_folder(cache_folder)

KeyboardInterrupt: 

## Load data

In [ ]:
# nb_link_ID_list = list(map(lambda x: x.ID, nb.link_list))

In [ ]:
# pickle.dump(nb_link_ID_list, open('nb_link_ID_list.pickle', 'wb'))

In [ ]:
# [m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list] = pickle.load(open('mc_data.pickle', 'rb'))

In [ ]:
# print len(observed_link_list)

## TEST DTA

In [ ]:
# ass_freq = nb.config.config_dict['DTA']['assign_frq']
# num_loading_interval = nb.config.config_dict['DTA']['max_interval'] * ass_freq

In [ ]:
ass_freq = 180
num_loading_interval = 24 * 180
num_path = 216720

In [ ]:
dta = mcdta_api()

In [ ]:
dta.initialize(cache_folder)

In [ ]:
dta.register_links(observed_link_list)

In [ ]:
# dta.register_paths(range(nb.config.config_dict['FIXED']['num_path']))
dta.register_paths(range(num_path))

In [ ]:
dta.install_cc()
dta.install_cc_tree()

In [ ]:
dta.run_whole()

In [ ]:
dta.get_car_link_tt(np.array([50, 100]))

In [ ]:
tt = dta.get_car_dar_matrix(np.arange(0, num_loading_interval, ass_freq), 
                np.arange(0, num_loading_interval, ass_freq) + ass_freq)

In [ ]:
tt

## Test MCDODE lib

In [ ]:
from mcDODE import *

In [ ]:
[m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list] = pickle.load(open('mc_data.pickle', 'rb'))

In [ ]:
data_dict = dict()
data_dict['car_count_agg_L_list'] = [L_car]
data_dict['truck_count_agg_L_list'] = [L_truck]
data_dict['car_link_flow'] = [m_car]
data_dict['truck_link_flow'] = [m_truck]
data_dict['car_link_tt'] = [m_spd_car]
data_dict['truck_link_tt'] = [m_spd_truck]

In [ ]:
config = dict()
config['use_car_link_flow'] = True
config['use_car_link_tt'] = False
config['use_truck_link_flow'] = False
config['use_truck_link_tt'] = False
config['car_count_agg'] = True
config['truck_count_agg'] = True
config['link_car_flow_weight'] = 1
config['link_truck_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 1
config['observed_links'] = observed_link_list
config['paths_list'] = range(nb.config.config_dict['FIXED']['num_path'])

In [ ]:
dode = MCDODE(nb, config)

In [ ]:
dode.add_data(data_dict)

In [ ]:
dode.estimate_path_flow(max_epoch = 100, step_size = 1e-4, 
                       car_init_scale = 0.01, truck_init_scale = 0.001,
                       store_folder = 'mcpath_storage')

In [ ]:
(f_car, f_truck) = pickle.load(open(os.path.join("mcpath_storage", '0iteration.pickle'), 'rb'))

In [ ]:
(p, link_seq, path_seq) = pickle.load(open(os.path.join('test.pickle'), 'rb'))

In [ ]:
p.shape

In [ ]:
num_assign_interval = 24
num_e_path = 216720
num_e_link = 2910
start_time = time.time()
a = coo_matrix((p, (link_seq, path_seq)), 
                   shape=(num_assign_interval * num_e_link, num_assign_interval * num_e_path))
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
a = a.tocsr()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
csr_matrix

## Compare

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from scipy import stats
def r2(predictions, targets):
    y_bar = np.mean(targets)
    # diff = np.minimum(np.abs(predictions - targets), targets)
    diff = predictions - targets
    ss_e = np.sum(diff ** 2)
    ss_t = np.sum((targets) ** 2)
    return 1 - ss_e / ss_t

def rsquared(x, y):
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    return r_value**2

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def rmsn(predictions, targets):
    return np.sqrt(np.sum((predictions - targets) ** 2) * len(predictions)) / np.sum(targets)

In [ ]:
[m_car, L_car, m_truck, L_truck, m_spd_car, m_spd_truck, observed_link_list] = pickle.load(
                    open('mc_data.pickle', 'rb'))
num_assign_interval = nb.config.config_dict['DTA']['max_interval']
ass_freq = nb.config.config_dict['DTA']['assign_frq']
num_loading_interval =  num_assign_interval * ass_freq

In [ ]:
dta = mcdta_api()
dta.initialize(cache_folder)

In [ ]:
dta.register_links(observed_link_list)
dta.register_paths(range(nb.config.config_dict['FIXED']['num_path']))

In [ ]:
dta.install_cc()
dta.install_cc_tree()

In [ ]:
dta.run_whole()

In [ ]:
x_e = dta.get_link_car_inflow(np.arange(0, num_loading_interval, ass_freq), 
                  np.arange(0, num_loading_interval, ass_freq) + ass_freq).flatten(order = 'F')
x_e = L_car.dot(x_e)

In [ ]:
x_e_nonana = x_e[~np.isnan(m_car)]
m_car_nonana = m_car[~np.isnan(m_car)]

In [ ]:
print r2_score(m_car_nonana, x_e_nonana)
print r2(m_car_nonana, x_e_nonana)
print rsquared(m_car_nonana, x_e_nonana)
print rmse(m_car_nonana, x_e_nonana)
print rmsn(m_car_nonana, x_e_nonana)

In [ ]:
x_e.shape

In [ ]:
m_car.shape

In [ ]:
plt.figure(figsize = (16,9), dpi = 300)
plt.plot(m_car_nonana, x_e_nonana, '*')
plt.show()